In [1]:
 # Use termcolor to make it easy to colorize the outputs.
from IPython import get_ipython
from IPython.display import display
# %%
 # Use termcolor to make it easy to colorize the outputs.
!pip install termcolor > /dev/null
!pip install langchain
!pip install openai
!pip install langchain_experimental
!pip install tiktoken
!pip install faiss-cpu==1.7.4
!pip install pydantic==2.9.2
from datetime import datetime, timedelta
from typing import List
import math
import faiss
import os
import logging
logging.basicConfig(level=logging.ERROR)
from langchain.cache import InMemoryCache # Import InMemoryCache
from langchain_experimental.generative_agents import GenerativeAgentMemory
from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS
from termcolor import colored
from langchain_experimental.generative_agents import (

    GenerativeAgent,
    GenerativeAgentMemory,
)

In [25]:
os.environ["OPENAI_API_KEY"] = ''

In [14]:
USER_NAME = "Buddy"  # The name you want to use when interviewing the agent.

LLM = ChatOpenAI(max_tokens=1500)  # Can be any LLM you want.

## Implementing Your First Generative Agent




In [15]:
def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OpenAIEmbeddings()
    # Initialize the vectorstore as empty
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    # Remove relevance_score_fn or define it before this line
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        # relevance_score_fn=relevance_score_fn,  # Remove this line if not needed
    )
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

In [16]:
from langchain_community.cache import BaseCache, InMemoryCache # Import BaseCache from langchain_community

cache = InMemoryCache()

alexis_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works,
    cache=cache # Pass the cache here
)

# Define Alexis

alexis = GenerativeAgent(
    name="Alexis",
    age=21,
    traits="curious, helpful",
    status="looking for a challenge",
    memory_retriever=alexis_memory.memory_retriever,
    llm=LLM,
    memory=alexis_memory,
)

# After initializing alexis_memory, call model_rebuild
alexis_memory.model_rebuild()



In [17]:
# The current "Summary" of a character can't be made because the agent hasn't made
# any observations yet.
print(alexis.get_summary())

Name: Alexis (age: 21)
Innate traits: curious, helpful
Alexis is a hardworking and dedicated individual who values honesty and integrity. She is confident in her abilities and takes pride in her work. Additionally, Alexis is a team player who is always willing to help others and collaborate on projects.


In [18]:
# We can add memories directly to the memory object

alexis_observations = [
    "Alexis recalls her morning walk in the park",
    "Alexis feels excited about the new book she started reading",
    "Alexis remembers her conversation with a close friend",
    "Alexis thinks about the painting she saw at the art gallery",
    "Alexis is planning to learn a new recipe for dinner",
    "Alexis is looking forward to her weekend trip",
    "Alexis contemplates her goals for the month."
]

for observation in alexis_observations:
    alexis.memory.add_memory(observation)



# We will see how this summary updates after more observations to create a more rich description.
print(alexis.get_summary(force_refresh=True))

Name: Alexis (age: 21)
Innate traits: curious, helpful
Alexis is reflective, goal-oriented, adventurous, appreciative of nature and art, and enjoys learning and trying new things.


## Interacting and Providing Context to Generative Characters

## Pre-Interview with Character

Before sending our character on their way, let's ask them a few questions.

In [19]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [20]:
interview_agent(alexis, "What do you like to do?")


'Alexis said "I enjoy reading, trying new recipes, going on adventures, and appreciating nature and art. How about you, Buddy?"'

## Step through the day's observations.

In [21]:
# Let's give Alexis a series of observations to reflect on her day
# Adding observations to Alexis' memory
alexis_observations_day = [
    "Alexis starts her day with a refreshing yoga session.",
    "Alexis spends time writing in her journal.",
    "Alexis experiments with a new recipe she found online.",
    "Alexis gets lost in her thoughts while gardening.",
    "Alexis decides to call her grandmother for a heartfelt chat.",
    "Alexis relaxes in the evening by playing her favorite piano pieces.",
]

for observation in alexis_observations_day:
    alexis.memory.add_memory(observation)


In [22]:
# Let's observe how Alexis's day influences her memory and character
for i, observation in enumerate(alexis_observations_day):
    _, reaction = alexis.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i + 1) % len(alexis_observations_day)) == 0:
        print("*" * 40)
        print(
            colored(
                f"After these observations, Alexis's summary is:\n{alexis.get_summary(force_refresh=True)}",
                "blue",
            )
        )
        print("*" * 40)


Alexis starts her day with a refreshing yoga session. Alexis smiles and feels grateful for starting her day on a positive note.
Alexis spends time writing in her journal. Alexis feels a sense of calm and satisfaction as she reflects on her thoughts and experiences in her journal.
Alexis experiments with a new recipe she found online. Alexis feels proud of herself for trying something new and looks forward to tasting the finished dish.
Alexis gets lost in her thoughts while gardening. Alexis feels a sense of peace and connection with nature as she loses herself in her thoughts while gardening.
Alexis decides to call her grandmother for a heartfelt chat. Alexis smiles and feels grateful for the opportunity to connect with her grandmother.
Alexis relaxes in the evening by playing her favorite piano pieces. Alexis feels a sense of peace and joy as she plays her favorite piano pieces.
****************************************
After these observations, Alexis's summary is:
Name: Alexis (age: 

# Adding Multiple Characters

In [26]:
cache = InMemoryCache()

henry_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works,
    cache=cache # Pass the cache here
)

# Define Henry

henry = GenerativeAgent(
    name="Henry",
    age=21,
    traits="curious, helpful",
    status="looking for a challenge",
    memory_retriever=henry_memory.memory_retriever,
    llm=LLM,
    memory=henry_memory,
)

# After initializing alexis_memory, call model_rebuild
henry_memory.model_rebuild()

In [27]:
# define
henry = GenerativeAgent(
    name="Henry",
    age=35,
    traits="gamer, foodie, tech guy",
    status="wants to work in startups ",
    memory_retriever=henry_memory.memory_retriever,
    llm=LLM,
    memory=henry_memory,
)

In [28]:
# Adding observations to Henry's Memories

henry_observations_day = [

    "Henry finished a challenging coding project last night",
    "Henry won a local gaming tournament over the weekend",
    "Henry tried a new sushi restaurant and loved it",
    "Henry read an article about the latest AI advancements",
    "Henry is planning a meetup with tech enthusiasts",
    "Henry discovered a bug in his latest app prototype",
    "Henry booked tickets for a tech conference next month",
    "Henry feels excited about a potential startup idea",
    "Henry spent the evening playing video games to unwind",
    "Henry is considering enrolling in a machine learning course"

]

for observation in henry_observations_day:
  henry.memory.add_memory(observation)

print(henry.get_summary())

Name: Henry (age: 35)
Innate traits: gamer, foodie, tech guy
Henry is a tech-savvy individual who is passionate about startups, gaming, coding, machine learning, and AI advancements. He is social and enjoys connecting with other tech enthusiasts, but also values relaxation and trying new experiences like new restaurants.


# Dialogue between Generative Agents

In [29]:
#Define conversations for agents to interact

def run_conversation(agents: List[GenerativeAgent], initial_observation: str) -> None:
    """Runs a conversation between agents."""
    _, observation = agents[1].generate_reaction(initial_observation)
    print(observation)
    max_turns = 3
    turns = 0
    while turns<=max_turns:
        break_dialogue = False
        for agent in agents:
            stay_in_dialogue, observation = agent.generate_dialogue_response(
                observation
            )
            print(observation)
            # observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                break_dialogue = True
        if break_dialogue:
            break
        turns += 1

In [30]:
agents = [alexis, henry]
run_conversation(agents,
                 "Alexis said: Hi, Henry how are you? I have been exploring how technology influences creativity. Since you are into tech, what do you think about this? "
                 )

Henry said "I think technology can definitely enhance creativity by providing new tools and platforms for expression."
Alexis said "I agree, Henry. Technology has opened up so many possibilities for creativity. Have you tried any new creative tools or platforms recently?"
Henry said "Yes, I've been experimenting with some new design software tools lately. It's amazing how technology can really push the boundaries of creativity."
Alexis said "That's so cool, Henry! Technology really does open up a whole new world of possibilities. I'm glad you're exploring new tools and pushing your creativity. Have you seen any particular improvements in your design work since using the new software?"
Henry said "Thanks, Alexis! I've definitely noticed an improvement in my design work. The new software has helped me streamline my process and create more polished designs. Have you tried any new tools or platforms that have had a similar impact on your work?"
Alexis said "That's great to hear, Henry! I h

# Lets see how the conversation went between the two agents

In [31]:
# Alexis
interview_agent(alexis, "How was your covnersation with Henry?")

'Alexis said "My conversation with Henry was great! We talked about technology, creativity, and how it can enhance our work. It was really inspiring. How about you, Buddy? Have you had any interesting conversations lately?"'

In [32]:
interview_agent(henry, "How was your conversation with Alexis?")

'Henry said "It was great! We always have such interesting discussions about technology and creativity. Alexis is always so insightful and supportive."'

----------------

# Dialogue Agent and Dialogue Simulator Classes

This approach chooses the next speaker and passes thier message to all agents and updates the conversation state.

In [37]:
from typing import Callable, List
import tenacity
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)

In [38]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

# Bidding Dialogue Agent class

In [40]:
from langchain.prompts import PromptTemplate

In [41]:
class BiddingDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,
        system_message: SystemMessage,
        bidding_template: PromptTemplate,
        model: ChatOpenAI,
    ) -> None:
        super().__init__(name, system_message, model)
        self.bidding_template = bidding_template

    def bid(self) -> str:
        """
        Asks the chat model to output a bid to speak
        """
        prompt = PromptTemplate(
            input_variables=["message_history", "recent_message"],
            template=self.bidding_template,
        ).format(
            message_history="\n".join(self.message_history),
            recent_message=self.message_history[-1],
        )
        bid_string = self.model([SystemMessage(content=prompt)]).content
        return bid_string